# FBN Scraping 

## Creating Folder to Save Materials

In [1]:
import os

# Specify the folder name
folder_name = "17.SP"

# Create the folder
os.makedirs(folder_name, exist_ok=True)

# Check if the folder was created
if os.path.exists(folder_name):
    print(f"Folder '{folder_name}' created successfully.")
else:
    print(f"Failed to create folder '{folder_name}'.")

Folder '17.SP' created successfully.


## Converting From PDF to Text

In [2]:
#Importing the needed library to convert PDF to text
import pdfplumber

#extracting the PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Replace 'your_pdf_file.pdf' with the actual path to your PDF file
pdf_path = r"C:\Users\salom\OneDrive\Área de Trabalho\FBN New\FBN\Issues\17.SP.pdf"

#variable containing text called extracted_text
extracted_text = extract_text_from_pdf(pdf_path)


In [3]:
#Downloading the text file 

with open('17.SP/17.SP_Issue_Text.txt', 'w', encoding='utf-8') as file:
    file.write(extracted_text)

## ISBNS

### Parsing ISBNs

In [4]:
#extracting ISBNs from thext 
#importing the library
import re

#defining the regex used 
def extract_isbns(text):
    # Define a regex pattern to match strings of numbers with dashes
    isbn_pattern = r"((978[-– ])?[0-9][0-9-– ]{10}[-– ][0-9xX])|((978)?[0-9]{9}[0-9Xx])"

    # Find all matches of the ISBN pattern in the text
    isbns = re.findall(isbn_pattern, text)

    return isbns

# Extract ISBNs from the example text
isbns_found = extract_isbns(extracted_text)

# Extract only the portions with numbers and dashes
isbns_found = [isbn[0] for isbn in isbns_found]

#print("Cleaned ISBNs:", isbns_found)


### ISBN Dataframe

In [5]:
#creating a dataframe that has ISBNs and the page numbers where found 

#importing necessary libaries
import pdfplumber
import pandas as pd
import re

def extract_isbns_from_text(text):
    # Define the ISBN regex pattern
    isbn_pattern = re.compile(r"((978[-– ])?[0-9][0-9-– ]{10}[-– ][0-9xX])|((978)?[0-9]{9}[0-9Xx])")

    # Find all ISBNs in the text
    isbns_found = isbn_pattern.findall(text)
    return isbns_found

def extract_text_from_pdf(pdf_path):
    data = {'PageNumber': [], 'Text': [], 'ISBNs': []}
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            # Add page number and corresponding text to the data dictionary
            data['PageNumber'].append(i)
            text = page.extract_text()
            data['Text'].append(text)
            
            # Extract ISBNs from the text
            isbns_found = extract_isbns_from_text(text)
            isbns_found = [isbn[0] for isbn in isbns_found]
            data['ISBNs'].append(isbns_found)

    return pd.DataFrame(data)

# Replace 'your_pdf_file.pdf' with the actual path to your PDF file
pdf_path = r"C:\Users\salom\OneDrive\Área de Trabalho\FBN New\FBN\Issues\17.SP.pdf"
df = extract_text_from_pdf(pdf_path)


In [6]:
df.head()

PageNumber                                               Text ISBNs
0           1  Feminist Bookstore News\nSource: Reveal Digita...    []
1           2  Feminist\nBookstore\nNews Iiv\nThe 1994 Sideli...    []
2           3  omplete\nSupport from\nWhen your customers are...    []
3           4  Feminist Bookstore News\nWelcome to FBN’s Tent...    []
4           5  Feminist Bookstore News\nI Am Becoming\nYou'll...    []

### Downloading List of ISBNs and Page Numbers

In [7]:
#change issue number
df.to_csv('17.SP/17.SP_text.csv', index=False)

## API

In [8]:
import requests
import csv
import time

api_key = '52084_9ddf62aeaa4a39b5485d9b7fb69dd5a8'
isbns_to_process = isbns_found  # Replace with the original list of ISBNs you want to process

# Removing Duplicates
isbns_to_process = list(set(isbns_to_process))

base_url = 'https://api2.isbndb.com/book/'

class RateLimiter:
    def __init__(self, calls_per_second):
        self.calls_per_second = calls_per_second
        self.interval = 1.0 / calls_per_second
        self.last_call = 0

    def wait(self):
        now = time.time()
        elapsed = now - self.last_call
        if elapsed < self.interval:
            time.sleep(self.interval - elapsed)
        self.last_call = time.time()

rate_limiter = RateLimiter(3)  # Allow 3 calls per second

with open('book_information.csv', 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['ISBN', 'Title', 'Author', 'Publisher', 'Pages', 'Date Published', 'Subjects', 'Binding', 'Synopsis', 'Language', 'Edition', 'Dimensions', 'MSRP', 'Image', 'Status']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    for isbn in isbns_to_process:
        url = f'{base_url}{isbn}'
        headers = {'Authorization': api_key}

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            book_info = response.json().get('book', {})
            writer.writerow({
                'ISBN': isbn,
                'Title': book_info.get('title', ''),
                'Author': ', '.join(book_info.get('authors', [])),
                'Publisher': book_info.get('publisher', ''),
                'Pages': book_info.get('pages', ''),
                'Date Published': book_info.get('date_published', ''),
                'Subjects': ', '.join(book_info.get('subjects', [])),  # Add subjects here
                'Binding': book_info.get('binding', ''),
                'Synopsis': book_info.get('synopsis', ''),
                'Language': book_info.get('language', ''),
                'Edition': book_info.get('edition', ''),
                'Dimensions': book_info.get('dimensions', ''),
                'MSRP': book_info.get('msrp', ''),
                'Image': book_info.get('image', ''),
                'Status': 'Success'
            })
        else:
            print(f"Error for ISBN {isbn}: {response.status_code}")
            writer.writerow({
                'ISBN': isbn,
                'Status': 'Error',
                'Title': '',
                'Author': '',
                'Publisher': '',
                'Pages': '',
                'Date Published': '',
                'Binding': '',
                'Synopsis': '',
                'Language': '',
                'Edition': '',
                'Dimensions': '',
                'MSRP': '',
                'Image': ''
            })

        rate_limiter.wait()  # Pause according to the rate limiter

print("CSV file created successfully.")

Error for ISBN : 403
Error for ISBN 6 - 8 65 55 1: 403
Error for ISBN 0-9624836-6-4: 403
Error for ISBN 8 5 554 706 9: 403
Error for ISBN 0-929999-03-7: 403
Error for ISBN 800-32 6-10 6: 403
Error for ISBN 0-89594-680-7: 403
Error for ISBN 0-89594-678-5: 403
Error for ISBN 10-682-5593 3: 403
Error for ISBN 0-89594-679-3: 403
Error for ISBN 1-56280-059-0: 403
Error for ISBN 1-883850-01-0: 403
Error for ISBN 1-56341-041-9: 403
Error for ISBN 1995 0-8351-2: 403
Error for ISBN 0-89594-551-7: 403
Error for ISBN 0-935155-15-5: 404
Error for ISBN 0-935155-16-3: 404
Error for ISBN 0-88961-191-2: 404
Error for ISBN 1-877920-05-3: 404
Error for ISBN 4 1 58-0303-1: 404
CSV file created successfully.


## Joining Dataframes

### Renaming to ISBN

In [9]:
import pandas as pd

# Assuming 'ISBNs' is the current column name and 'ISBN' is the desired column name
df.rename(columns={'ISBNs': 'ISBN'}, inplace=True)

### Adding Page Numbers

In [10]:
page = df[['PageNumber', 'ISBN']]
page.head()

PageNumber ISBN
0           1   []
1           2   []
2           3   []
3           4   []
4           5   []

In [11]:
page['ISBN'] = page['ISBN'].astype(str).str.replace('[\[\]\'\"]', '')
page.head()

C:\Users\salom\AppData\Local\Temp\ipykernel_22232\3003249529.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  page['ISBN'] = page['ISBN'].astype(str).str.replace('[\[\]\'\"]', '')
C:\Users\salom\AppData\Local\Temp\ipykernel_22232\3003249529.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  page['ISBN'] = page['ISBN'].astype(str).str.replace('[\[\]\'\"]', '')


PageNumber ISBN
0           1     
1           2     
2           3     
3           4     
4           5

In [12]:
# Split values on comma and explode to create new rows
page['ISBN'] = page['ISBN'].str.split(', ')
page = page.explode('ISBN')

# Display the updated DataFrame
page.head()

C:\Users\salom\AppData\Local\Temp\ipykernel_22232\1072530216.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  page['ISBN'] = page['ISBN'].str.split(', ')


PageNumber ISBN
0           1     
1           2     
2           3     
3           4     
4           5

### Opening Scraped Dataset

In [13]:


# Specify the file path
file_path = r'C:\Users\salom\OneDrive\Área de Trabalho\FBN New\FBN\notebooks\book_information.csv'

# Read the CSV file into a DataFrame
info = pd.read_csv(file_path)

info.head()

ISBN Title Author Publisher  Pages  Date Published  Subjects  \
0            NaN   NaN    NaN       NaN    NaN             NaN       NaN   
1  6 - 8 65 55 1   NaN    NaN       NaN    NaN             NaN       NaN   
2  0-9624836-6-4   NaN    NaN       NaN    NaN             NaN       NaN   
3  8 5 554 706 9   NaN    NaN       NaN    NaN             NaN       NaN   
4  0-929999-03-7   NaN    NaN       NaN    NaN             NaN       NaN   

  Binding  Synopsis Language Edition Dimensions  MSRP Image Status  
0     NaN       NaN      NaN     NaN        NaN   NaN   NaN  Error  
1     NaN       NaN      NaN     NaN        NaN   NaN   NaN  Error  
2     NaN       NaN      NaN     NaN        NaN   NaN   NaN  Error  
3     NaN       NaN      NaN     NaN        NaN   NaN   NaN  Error  
4     NaN       NaN      NaN     NaN        NaN   NaN   NaN  Error

### Joining it With Pages

In [14]:
merge = pd.merge(info, page, on='ISBN', how='left')
merge

ISBN            Title            Author Publisher  Pages  \
0             NaN              NaN               NaN       NaN    NaN   
1   6 - 8 65 55 1              NaN               NaN       NaN    NaN   
2   0-9624836-6-4              NaN               NaN       NaN    NaN   
3   8 5 554 706 9              NaN               NaN       NaN    NaN   
4   0-929999-03-7              NaN               NaN       NaN    NaN   
5   800-32 6-10 6              NaN               NaN       NaN    NaN   
6   0-89594-680-7              NaN               NaN       NaN    NaN   
7   0-89594-678-5              NaN               NaN       NaN    NaN   
8   10-682-5593 3              NaN               NaN       NaN    NaN   
9   0-89594-679-3              NaN               NaN       NaN    NaN   
10  1-56280-059-0              NaN               NaN       NaN    NaN   
11  1-883850-01-0              NaN               NaN       NaN    NaN   
12  1-56341-041-9              NaN               NaN       NaN    NaN   
13  1995 0-8351-2              NaN               NaN       NaN    NaN   
14  0-89594-551-7              NaN               NaN       NaN    NaN   
15  1-56280-065-5  Up, Up and Away  Ennis, Catherine  Naiad Pr  240.0   
16  0-935155-15-5              NaN               NaN       NaN    NaN   
17  0-935155-16-3              NaN               NaN       NaN    NaN   
18  0-88961-191-2              NaN               NaN       NaN    NaN   
19  1-877920-05-3              NaN               NaN       NaN    NaN   
20  4 1 58-0303-1              NaN               NaN       NaN    NaN   

    Date Published  Subjects    Binding  Synopsis Language        Edition  \
0              NaN       NaN        NaN       NaN      NaN            NaN   
1              NaN       NaN        NaN       NaN      NaN            NaN   
2              NaN       NaN        NaN       NaN      NaN            NaN   
3              NaN       NaN        NaN       NaN      NaN            NaN   
4              NaN       NaN        NaN       NaN      NaN            NaN   
5              NaN       NaN        NaN       NaN      NaN            NaN   
6              NaN       NaN        NaN       NaN      NaN            NaN   
7              NaN       NaN        NaN       NaN      NaN            NaN   
8              NaN       NaN        NaN       NaN      NaN            NaN   
9              NaN       NaN        NaN       NaN      NaN            NaN   
10             NaN       NaN        NaN       NaN      NaN            NaN   
11             NaN       NaN        NaN       NaN      NaN            NaN   
12             NaN       NaN        NaN       NaN      NaN            NaN   
13             NaN       NaN        NaN       NaN      NaN            NaN   
14             NaN       NaN        NaN       NaN      NaN            NaN   
15          1994.0       NaN  Paperback       NaN       en  First Edition   
16             NaN       NaN        NaN       NaN      NaN            NaN   
17             NaN       NaN        NaN       NaN      NaN            NaN   
18             NaN       NaN        NaN       NaN      NaN            NaN   
19             NaN       NaN        NaN       NaN      NaN            NaN   
20             NaN       NaN        NaN       NaN      NaN            NaN   

                                           Dimensions  MSRP  \
0                                                 NaN   NaN   
1                                                 NaN   NaN   
2                                                 NaN   NaN   
3                                                 NaN   NaN   
4                                                 NaN   NaN   
5                                                 NaN   NaN   
6                                                 NaN   NaN   
7                                                 NaN   NaN   
8                                                 NaN   NaN   
9                                                 NaN   NaN   
10                

### Adding a Column For Issue and Volume 

In [15]:
merge['Volume & Issue'] = '17.SP'

### Removing Duplicates

In [16]:
# Remove all duplicate rows from the DataFrame
merge = merge.drop_duplicates(keep=False)

# Reset the index after dropping duplicates
merge.reset_index(drop=True, inplace=True)

### Downloading Dataframe

In [17]:
# Assuming 'merge' is your DataFrame
merge.to_csv('17.SP/17.SP_booklist.csv', index=False)